In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

import torch
import torch.nn as nn

from sklearn.datasets import make_blobs

from sklearn.model_selection import train_test_split

In [ ]:
X,y = make_blobs(n_samples =4000, centers=2, n_features = 5, random_state = 13, cluster_std = 3)

In [ ]:
df = pd.DataFrame(X,columns=[f'feat_{i+1}' for i in range(5)])
df['Target'] = y
df.sample(6)

,feat_1,feat_2,feat_3,feat_4,feat_5,Target
1372,7.268641,-4.271387,5.071772,12.881156,7.658032,0
1099,5.448699,-3.074028,2.627482,9.245147,11.624333,0
357,8.065075,-5.161284,10.084474,12.107572,11.508151,0
1467,-3.099572,5.998640,6.705946,5.004809,2.771506,1
649,6.992758,-3.185975,4.559791,9.305803,8.102811,0
2600,4.070127,0.305646,8.968995,2.169626,-0.252477,1


In [ ]:
df['Target'].value_counts(normalize=True)

0    0.5
1    0.5
Name: Target, dtype: float64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=13, test_size=0.2)

In [ ]:
X_train = torch.from_numpy(X_train)
X_test  = torch.from_numpy(X_test)
Y_train = torch.from_numpy(y_train).view(-1,1).double()
Y_test = torch.from_numpy(y_test).view(-1,1).double()

X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

(torch.Size([3200, 5]),
 torch.Size([800, 5]),
 torch.Size([3200, 1]),
 torch.Size([800, 1]))

In [ ]:
X_train.dtype,X_test.dtype,Y_train.dtype

(torch.float64, torch.float64, torch.float64)

In [ ]:
class TabularClassification(nn.Module):

    def __init__(self,input_dim,output_dim):
        super(TabularClassification,self).__init__()

        self.fc  = nn.Linear(input_dim, output_dim, dtype= torch.float64)
        self.sig = nn.Sigmoid()

    def forward(self,x):

        out = self.fc(x)
        out = self.sig(out)

        return out

In [ ]:
model = TabularClassification(5,1)
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(),lr= 0.001)

for epochs in range(1000):


    model.train()
    output = model(X_train)    ## forward pass on train data
    loss   = criterion(output.view(-1,1),Y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_loss = loss.item()

    if epochs%10==0:

        model.eval()

        with torch.no_grad():
            outputs = model(X_test)
            valid_loss = criterion(outputs.view(-1,1),Y_test).item()

        print(f"Epochs {epochs}, Train loss = {train_loss}, validation loss = {valid_loss}")




Epochs 0, Train loss = 1.0121051028349566, validation loss = 1.0241107207849003
Epochs 10, Train loss = 0.9606195079480088, validation loss = 0.9739906233450967
Epochs 20, Train loss = 0.9126623563091201, validation loss = 0.9272167450421844
Epochs 30, Train loss = 0.8680123368376697, validation loss = 0.8836029613412592
Epochs 40, Train loss = 0.8264767245197474, validation loss = 0.8429809655778125
Epochs 50, Train loss = 0.7878761483774808, validation loss = 0.8051887593764782
Epochs 60, Train loss = 0.7520368386693105, validation loss = 0.7700654764960396
Epochs 70, Train loss = 0.7187874612434428, validation loss = 0.7374499043685917
Epochs 80, Train loss = 0.6879586391758162, validation loss = 0.7071810340436088
Epochs 90, Train loss = 0.6593838846374507, validation loss = 0.6790995736475495
Epochs 100, Train loss = 0.6329010983205395, validation loss = 0.653049759288381
Epochs 110, Train loss = 0.6083541175081801, validation loss = 0.628881070896405
Epochs 120, Train loss = 0.58